# Aprendizaje por refuerzo

## Q-Learning

En el algoritmo Q-Learning, La "Q" de su nombre proviene de *quality* y tiene su razón de ser porque cada valor de la tabla $Q$ indica la calidad de cada acción para llegar a una recompensa futura.

## Ejemplo

Supongamos que tenemos un robot cuyo objetivo es aprender a salir de una casa, como la que muestra el plano de la figura siguiente. Para ello, va a realizar una serie de múltiples intentos, obteniendo recompensa únicamente cuando consiga salir. En cada intento el robot partirá desde alguna habitación aleatoria. A estos "intentos" le daremos el nombre de "episodios". Denominaremos **episodio** al conjunto de acciones que el robot toma desde que parte inicialmente de una habitación hasta que consigue salir.

<img src="imgs/plano.png" width="40%">

Las puertas que dan directamente al exterior tienen una recompensa de 100. El resto de puertas no tienen recompensa. El plano de la casa puede ser visto como un grafo (figura siguiente). Cuando el robot llega al estado número 5 del grafo el intento se da por finalizado.

<img src="imgs/grafo.png" width="40%">

Este grafo puede ser representado también por una matriz donde las filas representan los estados y las columnas las acciones que se pueden tomar. En este caso en particular, las acciones corresponden a los estados a los que se puede ir. Así que, en este caso, la matriz es cuadrada.
Vamos a llamar a esta matriz $R$, **matriz de recompensas**.  El valor $-1$ significa que una determinada acción no es posible para un determinado estado.


$$R = \begin{pmatrix}
-1 & -1 & -1 & -1 & 0 & -1 \\
-1 & -1 & -1 & 0 & -1 & 100 \\
-1 & -1 & -1 & 0 & -1 & -1 \\
-1 & 0 & 0 & -1 & 0 & -1 \\
0 & -1 & -1 & 0 & -1 & 100 \\
-1 & 0 & -1 & -1 & 0 & 100
\end{pmatrix}$$

Este algoritmo de aprendizaje por refuerzo almacena en una tabla el conocimiento que va adquiriendo, que llamaremos **tabla $Q$**. Inicialmente estará vacía. 

$$Q = \begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0
\end{pmatrix}$$


A medida que el robot vaya deambulando por la casa y completando episodios irá acumulando recompensas. 

Supongamos que el robot parte de la habitación (estado) 1. En ese momento, el robot no tiene ningún tipo de conocimiento de la casa, no sabe qué puerta elegir para llegar antes a la salida. Por supuesto, no tiene acceso a la matriz de recompensas. En esas condiciones, el robot solo puede hacer una elección aleatoria de una de las dos puertas, supongamos que elige la inferior (véase el plano de la casa). Es decir, elige la acción “ir al estado 3”. 

Una vez en la habitación 3 descubre que no recibe ninguna recompensa y que vuelve a encontrarse en la misma situación, ¿qué puerta elegir? Todas, incluso ir de nuevo al estado 1, son para el robot elecciones aceptables, puesto que todas le proporcionan la misma incertidumbre. De nuevo, mediante una selección totalmente aleatoria, selecciona la puerta izquierda, “ir al estado 4”.

Ya en el estado 4 la situación se repite. De nuevo, no recibe ninguna recompensa. Otra vez de forma aleatoria, elige la puerta inferior “ir al estado 5”.

Llegado al estado 5, el robot descubre que recibe 100 puntos (puntos, dinero, gallifantes... cualquier cosa vale) de recompensa. En ese momento el robot actualizará su tabla $Q$, dado que hay algo de información nueva. Actualizará, por tanto, la entrada $(4,5)$ con el valor $100$. Démonos cuenta de que $4$ representa el estado en el que se encontraba el robot y $5$ es la acción que tomó ("ir al estado 5"). En otras palabras significaría que debemos apuntar en una libreta que si estamos en la habitación 4 y vamos por la puerta inferior obtendremos 100 puntos de recompensa. La próxima vez que estemos en la habitación $4$, ya sabremos qué elegir.


Finalmente, como el robot ya ha salida de la casa, el episodio termina.

La tabla $Q$ actualizada será:

$$Q = \begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 100 \\
0 & 0 & 0 & 0 & 0 & 0
\end{pmatrix}$$



Empezamos, por tanto, un nuevo episodio. Supongamos que, por una cuestión aleatoria, el robot parte de la habitación 3. Y, de nuevo, el azar nos lleva a seleccionar la puerta de la izquierda, (ir al estado 4).
Tengo que confesarte que, por simplificar, en el episodio anterior no expliqué completamente cómo se actualiza la tabla $Q$. Ahora sí que la vamos a ir actualizando correctamente. Para ello, vamos a hacer uso de esta fórmula, denominada **ecuación de Bellman**:

$$Q(s,a) = R(s,a) + \gamma max[Q(s',a')]$$

Significa lo siguiente, cuando el robot se encuentra en el estado $s$ y toma la acción $a$ pasa al estado $s’$. Una vez en el estado $s’$ podemos consultar la tabla $Q$ para ver qué acción $a’$ es la que tiene la recompensa máxima, $max[Q(s’,a’)]$. Por tanto, la actualización de $Q(s,a)$ se compone de dos partes. En primer lugar, la recompensa directa por haber pasado de $s$ a $s’$ mediante la acción $a$, que en este caso es $R(3,4) = 0$. Y, en segundo lugar, la recompensa máxima que se puede obtener desde $s’$ tomando la acción $a’$ adecuada. El factor $\gamma$ debe tener un valor mayor que $0$ y menor que $1$, pongámosle $0.8$. Su cometido es rebajar proporcionalmente la recompensa que está dos pasos más allá del estado $s$. Por tanto, la nueva actualización de $Q(3,4)$ será:

$$Q(3,4) = R(3,4) + \gamma \cdot max[Q(4,0),Q(4,3),Q(4,5)]$$

Que es:

$$Q(3,4) = 0 + 0.8 \cdot max[0, 0, 100]$$

Los nuevos valores de $Q$ serán:

$$Q = \begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 80 & 0 \\
0 & 0 & 0 & 0 & 0 & 100 \\
0 & 0 & 0 & 0 & 0 & 0
\end{pmatrix}$$

Por tanto, ¿qué almacena la tabla $Q$? Realmente, no almacena recompensas, sino información. Los valores de la tabla $Q$ nos dan una referencia sobre la cercanía o lejanía a la que la recompensa real está.

Aún no hemos terminado este segundo episodio. Nos encontramos en el estado 4. Si el robot consulta la tabla $Q$ puede ver que si elige la acción “ir al estado 5” obtendrá mayor recompensa que si toma cualquiera de las otras opciones, que, por el momento, están a $0$. Supongamos que elige “ir al estado 5” y el episodio termina.

Comencemos con el tercer episodio. El robot parte de la habitación 1 (por azar). Si escoge la acción “ir al estado 3” deberá actualizar la tabla $Q$ de la siguiente forma:

$$Q(1,3) = R(1,3) + \gamma \cdot max[Q(3,1), Q(3,2), Q(3,4)]$$

Lo cual es:

$$Q(1,3) = 64  = 0 + 0.8 \cdot max[0, 0, 80] $$

Con lo que la tabla $Q$ quedaría:

$$Q = \begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 64 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 80 & 0 \\
0 & 0 & 0 & 0 & 0 & 100 \\
0 & 0 & 0 & 0 & 0 & 0
\end{pmatrix}$$

A partir de aquí el robot podría ir utilizando la información de la tabla $Q$ para guiar su toma de decisiones, pasando del estado $3$ al $4$ y del $4$ al $5$, finalizando el tercer episodio.


Supongamos ahora que en un cuarto episodio el robot parte de nuevo desde la habitación 1. Si se guía por la información de la tabla $Q$ podría llegar a la salida en tres pasos. Sin embargo, si elige la salida superior (por supuesto, la puerta está cerrada y no sabe a dónde lleva) llegaría a la salida en un solo paso, lo cual es mucho mejor. A medida que el robot recaba nueva información, puede reutilizarla en episodios posteriores, es decir, puede **explotar** la información que ya tiene. O, por el contrario, puede aventurarse a descubrir nuevos caminos, es decir, **explorar** nuevas vías que, en ocasiones, pueden llevarle a mucho mejores resultados.




### Explotación vs. exploración

El algoritmo Q-Learning debe acompasar una estrategia que combine cierta explotación con cierta exploración. Al principio, es evidente que lo único que se puede hacer es explorar, puesto que nuestra tabla 𝑄Q está vacía, no hay información. Pero, a medida que vamos completando episodios, debemos explotar esta información para obtener recompensas seguras.

Al tipo de combinación que hagamos sobre exploración y explotación es lo que se denomina **política**.

### Convergencia

¿Cuándo termina el algoritmo Q-Learning? La primera respuesta sería: cuando la tabla $Q$ converja. Esto significa que cuando hayamos hecho los suficientes episodios, la tabla $Q$ ya no modificará más sus valores, a esta tabla la llamaremos $Q^*$. Esto ocurre fácilmente en casos como el de nuestro ejemplo. Pero en casos complejos, la tabla puede ser muy grande y sería necesario mucho tiempo (más del disponible) para que la tabla llegue a converger. Por tanto, la segunda respuesta es que no termina nunca. Siempre se estará ejecutando una determinada política que alterne, de la manera más eficiente posible, explotación y exploración.

### Implementación del algoritmo

Establecemos los parámetros del algoritmo

In [1]:
import random

discount = 0.8 # gamma
learning_rate = 0.5 # alfa
final_state = 5

Inicializamos la tabla de recompensas

In [2]:
rewards = [[-1., -1., -1., -1., 0., -1.],
           [-1., -1., -1., 0., -1., 100.],
           [-1., -1., -1., 0., -1., -1.],
           [-1., 0., 0., -1., 0., -1.],
           [0., -1., -1., 0., -1., 100.],
           [-1., 0., -1., -1., 0., 100.]]

Inicializamos la tabla $Q$ a cero o cualquier valor.

In [6]:
Q = [[0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0.]]

import random
        
from tabulate import tabulate
print(tabulate(Q, tablefmt="grid"))

+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+


Fórmula de actualización de la matriz $Q$ 

$$Q(s,a) = R(s,a) + \gamma max[Q(s',a')]$$

In [7]:
def qlearning1(s, a):
    Q[s][a] = rewards[s][a] + discount * max(Q[a])
    return

In [9]:
for _ in range(100):

    s = random.randint(0, 5)
    while s == final_state:
        s = random.randint(0, 5)

    keep = True
    while keep:
        a = random.randint(0, 5)
        while rewards[s][a] == -1:
            a = random.randint(0, 5)
        qlearning1(s, a)
        s = a
        if s == final_state:
            keep = False 
            
print(tabulate(Q, tablefmt="grid"))

+----+----+------+----+----+-----+
|  0 |  0 |  0   |  0 | 80 |   0 |
+----+----+------+----+----+-----+
|  0 |  0 |  0   | 64 |  0 | 100 |
+----+----+------+----+----+-----+
|  0 |  0 |  0   | 64 |  0 |   0 |
+----+----+------+----+----+-----+
|  0 | 80 | 51.2 |  0 | 80 |   0 |
+----+----+------+----+----+-----+
| 64 |  0 |  0   | 64 |  0 | 100 |
+----+----+------+----+----+-----+
|  0 |  0 |  0   |  0 |  0 |   0 |
+----+----+------+----+----+-----+


### Ejercicios prácticos

* Actualiza la tabla $Q$ realizando a mano un episodio con el siguiente recorrido $0\rightarrow4\rightarrow3\rightarrow1\rightarrow5$.


* Crea una función que, a partir de la matriz $Q^*$, nos lleve a la salida por el camino óptimo desde cualquier habitación.


* ¿Qué pasaría si el factor $\gamma$ fuera $1$?
